<a href="https://colab.research.google.com/github/KoichiFunaya/Colab/blob/dev/News_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Transformers Library**

In [1]:
!pip install transformers

# **Load Libraries**

In [2]:
import numpy as np
import pandas as pd
import configparser
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast

## **Load Utility Class Instance**

In [3]:
class NewsAnalyzer:
    # annoate patterns

    def __init__(self):
        
        # decide in which environment you are in
        try:
            # first try google.colab
            from google.colab import drive
            self.in_colab = True
            print("in Google COLAB: load Google Drive and use GPU")
            self.hostname = "GOOGLE_COLAB"
            # mount Google Drive
            drive.mount('/content/drive')
            # switch to the github folder
            % cd /content/drive/MyDrive/Colab\ Notebooks/colab
            # pull the gitlab, to make sure it's up to date
            ! git pull origin
        except:
            # we are not in google.colab
            print("not in Google COLAB: access the data folder and use CPU")
            self.in_colab = False
            # check what the hostname is
            import socket
            self.hostname=socket.gethostname()
            print("hostname = {}".format(self.hostname))

        # load the configuration file "config.ini"
        self.config = configparser.ConfigParser()
        self.config.read("config.ini")
        # set the path parameters
        self.data_dir    = self.config[self.hostname]['data_dir']
        self.sample_dir  = self.config[self.hostname]['sample_dir']
        self.model_path  = self.config[self.hostname]['model_path']
        self.label_path  = self.config[self.hostname]['label_path']
        
        pass
    
        
    def load_device(self):
        
        # load processor device
        if self.in_colab:
            # load CUDA
            assert(torch.cuda.is_available())
            self.has_cuda = True
            self.device = torch.device("cuda")
        else:
            try:
                # check if CUDA device exists
                assert(torch.cuda.is_available())
                self.has_cuda = True
            except:
                # CUDA device does not exist
                self.has_cuda = False
            # in either case, we don't load CUDA, just use CPU.
            self.device = torch.device("cpu")
            
        return self.device, self.in_colab
    
    
    def load_labelled_dataset(self):
            
        self.sent_df = pd.read_excel(self.label_path,sheet_name=self.config['LABEL_DATASET']['label_sheet'])
            
        return self.sent_df

In [4]:
# create the utility class instance
_NewsAnalyzer = NewsAnalyzer()  

not in Google COLAB: access the data folder and use CPU
hostname = N-1274


## **Load device and dataset**

In [5]:
DEVICE,IN_COLAB = _NewsAnalyzer.load_device()
DATA_DIR   = _NewsAnalyzer.data_dir
MODEL_PATH = _NewsAnalyzer.model_path

In [6]:
df = _NewsAnalyzer.load_labelled_dataset()
df.head()

,Sentence,Date,Subject,Object,Predicate,Sentiment,Source,Time Annotated,Note
0,中国はこれまで長期にわたり、このような重要な新興分野で至急の導入が必要とされるハイレベルかつ...,2021-03-19 08:46:18.810,中国,苦境,立たす,0,D:\Personal\Koichi\DATA\mvp21\news\afpbb,2021-03-21 21:07:48.290,NaN
1,キーテクノロジーを他国に握られることによる需給の矛盾が日増しに顕著になっている。,2021-03-19 08:46:18.810,他国,キーテクノロジーを,握る,0,D:\Personal\Koichi\DATA\mvp21\news\afpbb,2021-03-21 21:09:06.837,受動態をあたかも能動態である可能ように扱っているので、間違い。
2,このような背景の中、中国国産の「手で裂けるステンレス」の登場は、産業チェーンの不足を補い、効...,2021-03-19 08:46:18.810,中国,関連分野における中国の自主コントロール能力を,向上,0,D:\Personal\Koichi\DATA\mvp21\news\afpbb,2021-03-21 21:10:19.541,「中国」という話題を記述できるようにすること。
3,\n\n今日の中国は、世界一の製造大国としての地位がいっそう強固となった。,2021-03-19 08:46:18.810,中国,世界一の製造大国としての地位が,強固,0,D:\Personal\Koichi\DATA\mvp21\news\afpbb,2021-03-21 21:11:38.487,ROOT
4,こうしたデータの1つ1つは、中国が製造強国の建設を加速させる十分な底力と強い自信となった。,2021-03-19 08:46:18.810,中国,自信,なる,0,D:\Personal\Koichi\DATA\mvp21\news\afpbb,2021-03-21 21:12:49.857,主語である「中国」が存在しない


In [7]:
df.shape

(161, 9)

In [9]:
# check class distribution
df['Sentiment'].value_counts(normalize = True)

 0    0.472050
-1    0.440994
 1    0.086957
Name: Sentiment, dtype: float64

## **Split train dataset into train, validation and test sets**

In [11]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df['Sentence'], df['Sentiment'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df['Sentiment'])

# we will use temp_text and temp_labels to create validation and test set
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

## **Import BERT Model and BERT Tokenizer**

In [ ]:
# import BERT-base pretrained model
bert = AutoModel.from_pretrained('bert-base-uncased')

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
# sample data
text = ["this is a bert model tutorial", "we will fine-tune a bert model"]

# encode text
sent_id = tokenizer.batch_encode_plus(text, padding=True, return_token_type_ids=False)

In [ ]:
# output
print(sent_id)

# **Tokenize the Sentences**

In [ ]:
# get length of all the messages in the train set
seq_len = [len(i.split()) for i in train_text]

pd.Series(seq_len).hist(bins = 30)

In [ ]:
max_seq_len = 25

In [ ]:
# tokenize and encode sequences in the training set
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length = max_seq_len,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),
    max_length = max_seq_len,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

# tokenize and encode sequences in the test set
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(),
    max_length = max_seq_len,
    padding='max_length',
    truncation=True,
    return_token_type_ids=False
)

## **Convert Integer Sequences to Tensors**

In [ ]:
## convert lists to tensors

train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

## **Create DataLoaders**

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 32

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

## **Freeze BERT Parameters¶**

In [ ]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False

## **Define Model Architecture**

In [ ]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
      super(BERT_Arch, self).__init__()

      self.bert = bert 
      
      # dropout layer
      self.dropout = nn.Dropout(0.1)
      
      # relu activation function
      self.relu =  nn.ReLU()

      # dense layer 1
      self.fc1 = nn.Linear(768,512)
      
      # dense layer 2 (Output layer)
      self.fc2 = nn.Linear(512,2)

      #softmax activation function
      self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

      #pass the inputs to the model  
      _outp  = self.bert(sent_id, attention_mask=mask)
      cls_hs = _outp.pooler_output
      
      x = self.fc1(cls_hs)

      x = self.relu(x)

      x = self.dropout(x)

      # output layer
      x = self.fc2(x)
      
      # apply softmax activation
      x = self.softmax(x)

      return x

In [ ]:
# pass the pre-trained BERT to our define architecture
model = BERT_Arch(bert)

In [ ]:
# push the model to GPU
model = model.to(device)

In [ ]:
# optimizer from hugging face transformers
from transformers import AdamW

# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

## **Find Class Weights**

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

#compute the class weights
class_weights = compute_class_weight('balanced', np.unique(train_labels), train_labels)

print("Class Weights:",class_weights)

In [ ]:
# convert class weights to tensor
weights= torch.tensor(class_weights,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights) 

# number of training epochs
epochs = 10

## **Fine-Tune BERT**

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

## **Start Model Training**

In [ ]:
from timeit import default_timer as timer
from datetime import timedelta

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

# measure time
_start = _t0 = timer()

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), MODEL_PATH)
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

        # show the elapsed time
    _t1 = timer()
    print('Elapsed time: {}'.format(timedelta(seconds=_t1-_t0)))
    _t0 = _t1
    
# show the total time for the training
_t1 = timer()
print('\nTotal time elapsed: {}'.format(timedelta(seconds=_t1-_start)))

## **Load Saved Model**

In [ ]:
#load weights of best model
model.load_state_dict(torch.load(MODEL_PATH))

## **Get Predictions for Test Data**

In [ ]:
# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [ ]:
# model's performance
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

In [ ]:
# confusion matrix
pd.crosstab(test_y, preds)

In [ ]:
if IN_COLAB:
    if input('OK to unmount drive? (y/n)')=='y':
        drive.flush_and_unmount()
        print('All changes made in this colab session should now be visible in Drive.')